In [ ]:
import sys
import os
import tqdm
import copy

import numpy as np
import matplotlib.pyplot as plt


sys.path.insert(0, os.path.abspath("../cli"))
os.environ["PATH"] = '/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/comm_libs/mpi/bin:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/compilers/bin:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/cuda/bin:'+os.environ["PATH"]
os.environ["LD_LIBRARY_PATH"] = '/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/comm_libs/nvshmem/lib:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/comm_libs/nccl/lib:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/comm_libs/mpi/lib:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/math_libs/lib64:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/compilers/lib:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/cuda/lib64:'+os.environ["LD_LIBRARY_PATH"]
os.environ["CPATH"] = '/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/comm_libs/nvshmem/include:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/comm_libs/nccl/include:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/comm_libs/mpi/include:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/math_libs/include:/home/shared/software/cuda/hpc_sdk-centos8/Linux_x86_64/23.1/cuda/include'#+os.environ["CPATH"]
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from simulation.sonar import Sonar
from simulation.utils import (
    FlatBottom,
    CircleBottom,
    EllipsisBottom,
    positions_line,
    positions_half_circle,
)
from simulation.plotting import plot_velocity, plot_snapshot_and_signal
from simulation.sources import GaborSource 
from devito import configuration

plt.rcParams["figure.figsize"] = (10, 10)
configuration['log-level'] = 'DEBUG'
configuration['language'] = 'openacc'
configuration['platform'] = 'nvidiaX'
configuration['compiler'] = 'nvc'
configuration['mpi'] = False

### Function

In [ ]:
#run a simulation in an idealised scenario - only as long as the wave does not hit the wall.
#TODO: would be nicer to get an arrat of posy, and return an array of results
def get_ideal_signal(
    domain_size,
    f0,
    v_env,
    sd,
    ns,
    posy,
    angle,
    dt,
    spatial_dist,
    space_order,
    plot=False,
):
    max_distance = domain_size[1]
    t_end = max_distance / v_env
    s = Sonar(
        domain_size,
        f0,
        v_env,
        CircleBottom(domain_size[0] / 2, domain_size[1] / 2, domain_size[1] / 2),
        dt=dt,
        tn=t_end,
        spatial_dist=spatial_dist,
        space_order=space_order,
    )
    src_coord = np.array(
        [(domain_size[0] - sd * ns) / 2, domain_size[1] / 2]
    ) + positions_line(stop_x=ns * sd, posy=sd, n=ns)
    
    src_args = {
        "name": "src",
        "grid": s.model.grid,
        "npoint": ns,
        "f0": f0,
        "time_range": s.time_range,
        "coordinates_data": src_coord
    }
    distance = domain_size[1] / 2 * posy
    rec_x = domain_size[0] / 2 - np.cos(np.deg2rad(angle)) * distance
    rec_y = domain_size[1] / 2 + np.sin(np.deg2rad(angle)) * distance
    rec_args = {
        "name": "rec",
        "grid": s.model.grid,
        "time_range": s.time_range,
        "npoint": 1,
        "coordinates": np.array([[rec_x, rec_y]])
    }
    s.set_source("GaborSource", src_args)
    s.set_receiver("Receiver", rec_args)
    s.finalize()
    s.run_beam(angle)
    if plot:
        plot_velocity(
            s.model, source=s.src.coordinates.data, receiver=s.rec.coordinates.data
        )
        plt.plot(s.rec.data)
    return s.rec.data[:, 0]

### Beamforming

In [ ]:
# Simplest example - water with no object, beams at a few angles
v_env = 1.5  # speed of sound - 1.5 km/s
domain_size = (60, 60)  # 60m by 60m domain
f0 = 50  # 50kHz source
angles = [45, 60, 75, 90, 105, 120, 135]
space_order = 8
spatial_dist = round(v_env / f0 / 3, 6)
dt = spatial_dist / 20  # TODO: check if this is not too small?
sonar = Sonar(
    domain_size,
    f0,
    v_env,
    FlatBottom(),
    space_order=space_order,
    dt=dt,
    spatial_dist=spatial_dist,
)

In [ ]:
ns = 128 #number of sources
source_distance = 0.002 #separated by 2mm
cy = (ns - 1) / 2 * source_distance + source_distance #distance from top

src_coord = np.array(
    [(domain_size[0] - source_distance * ns) / 2, cy]
) + positions_line(stop_x=ns * source_distance, posy=source_distance, n=ns)

src_args = {
    "name": "src",
    "grid": sonar.model.grid,
    "npoint": ns,
    "f0": f0,
    "time_range": sonar.time_range,
    "coordinates": src_coord,
}

rec_args = {
    "name": "rec",
    "grid": sonar.model.grid,
    "time_range": sonar.time_range,
    "npoint": 180,
    "coordinates": positions_half_circle(domain_size[0] / 2, domain_size[1] / 2, cy, 180),
}

sonar.set_source("GaborSource", src_args)
sonar.set_receiver("Receiver", rec_args)
sonar.finalize()


In [ ]:
print(
    f"The computational grid has {sonar.model.grid.shape} grid points and physical size of {sonar.model.grid.extent} m"
)
print(f"The time step is {sonar.model.critical_dt}")
print(f"fs: {1 / sonar.model.critical_dt}")
print(f"Spacing: {sonar.model.spacing_map}")
print(f"Time range: {sonar.time_range.num}")

In [ ]:
plot_velocity(
    sonar.model,
    source=sonar.src.coordinates.data,
    receiver=sonar.rec.coordinates.data,
)

plt.figure()
plt.plot(sonar.src.time_values[:800], sonar.src.signal_packet)
plt.xlabel("Time (ms)")
plt.ylabel("Amplitude")
ax2 = plt.gca().twiny()
ax2.set_xlim(0, 800)
ax2.set_xlabel("Timesteps")
plt.title("Emitted signal (Gabor wavelet)")
plt.tight_layout()
plt.show()

In [ ]:
recordings = {}
for a in angles:
    sonar.run_beam(a)
    recordings[a] = copy.deepcopy(sonar.rec.data)

In [ ]:
#Now we plot attenuation profiles and received signals
fig, ax = plt.subplots(2, 7, figsize=(35, 15), gridspec_kw={"height_ratios": [1, 2]})
plt.title("Attenuation profiles (top row), received signals (bottom row)")
for i, (a, r) in enumerate(recordings.items()):
    recording = np.flip(r)
    ax[0, i].plot(
        np.max(np.abs(recording[:, :]), axis=0) / np.max(np.abs(recording[:, :]))
    )
    ax[0, i].set_xlabel("Degree (Receivers)")
    ax[0, i].set_ylabel("Attenuation")
    ax[0, i].set_title(f"{a}°")
    ax[0, i].tick_params()

    ax[1, i].plot(r[40000:42000, 180 - a])
    ax[1, i].set_xlabel("Timestep")
    ax[1, i].set_ylabel("Amplitude")
fig.tight_layout()
plt.savefig("output/beamforming.pdf", bbox_inches="tight", dpi=600, format="pdf")

### Ideal signal

In [ ]:
domain_size = (20, 20)
distances = [0.25, 0.5, 0.75]
f0 = 50
v_env = 1.5
space_order = 8
spatial_dist = round(v_env / f0 / 3, 6)
dt = spatial_dist / 20
source_distance = 0.002


In [ ]:
ideal_signals = {}
angles = [45, 90]
for a in angles:
    for d in distances:
        ideal_signals[(a, d)] = get_ideal_signal(
            domain_size,
            f0,
            v_env,
            source_distance,
            ns,
            d,
            a,
            dt,
            spatial_dist,
            space_order,
            plot=False,
        )

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20, 12))
for i, angle in enumerate(angles):
    fig.suptitle(f"Ideal signals for angles {angles}")

    # Plot each distance for the current angle
    axs[i, 0].plot(ideal_signals[(angle, 0.25)][3400:4400])
    axs[i, 0].set_title(f"Distance: {distances[0] * 20 / 2} m")

    axs[i, 1].plot(ideal_signals[(angle, 0.5)][6700:7700])
    axs[i, 1].set_title(f"Distance: {distances[1] * 20 / 2} m")

    axs[i, 2].plot(ideal_signals[(angle, 0.75)][10000:11000])
    axs[i, 2].set_title(f"Distance: {distances[2] * 20 / 2} m")

    # Plot the original signal in the last column
    axs[i, 3].plot(sonar.src.data[:1000, 0])
    axs[i, 3].set_title(f"Original signal")

fig.tight_layout()
plt.savefig("output/ideal_signals_combined.pdf", bbox_inches="tight", dpi=600, format="pdf")

### Detection with correlation

In [ ]:
domain_size = (60, 30)
radius = 28
v_env = 1.5
ns = 128
source_distance = 0.002
cy = (ns - 1) / 2 * source_distance + source_distance
f0 = 50
space_order = 8
spatial_dist = round(v_env / f0 / 3, 5)
dt = spatial_dist / 20


In [ ]:
sonar = Sonar(
    domain_size,
    f0,
    v_env,
    EllipsisBottom(True),
    space_order=space_order,
    dt=dt,
    spatial_dist=spatial_dist,
)
sonar.set_source()
sonar.set_receiver()
sonar.finalize()

In [ ]:
plot_velocity(
    sonar.model, source=sonar.src.coordinates.data, receiver=sonar.rec.coordinates.data
)

In [ ]:
ideal_signal = sonar.src.signal_packet
plt.plot(ideal_signal)
plt.title("Transmitted signal")
plt.show()


In [ ]:
sonar.run_beam(45)
recording_45 = copy.deepcopy(sonar.rec.data)

In [ ]:
# The emitted signal on receivers
plt.plot(recording_45[:5000, 64])
start_time = np.argmax(recording_45[:5000, 64])


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].plot(ideal_signal)
ax[1].plot(recording_45[5000:, 64])
plt.savefig("output/correlation_ideal_recording.pdf", bbox_inches="tight", dpi=600, format="pdf")

In [ ]:
correlate = np.correlate(recording_45[5000:, 64], ideal_signal, mode="same")
plt.figure(figsize=(20, 15))
plt.plot(correlate)
plt.show()
print(np.argmax(correlate))

In [ ]:
peak = 5000 + correlate.argmax()
## change the figure size
plt.figure(figsize=(20, 15))
plt.plot(recording_45[:, 64])
plt.plot(peak, recording_45[peak, 64], "o")
plt.show()
plt.savefig("output/correlate.pdf", bbox_inches="tight", dpi=600, format="pdf")

In [ ]:
#TODO: this does not seem right
distance = (peak - start_time) * sonar.model.critical_dt * v_env / 2
print(f"Distance: {distance} m")


### Detect multiple angles

In [ ]:
domain_size = (60, 30)
v_env = 1.5
ns = 128
source_distance = 0.002
f0 = 50
space_order = 8
spatial_dist = round(v_env / f0 / 3, 5)
dt = spatial_dist / 20
angles = [30, 45, 60, 75, 90, 105, 120, 135, 150]


In [ ]:
obstacle = True
v_wall = 5.64
v_obj = 3.24
domain_dims = (
    round(domain_size[0] / spatial_dist),
    round(domain_size[1] / spatial_dist),
)
vp = np.full(domain_dims, v_env, dtype=np.float32)
r_obs = vp.shape[0] / 20
a, b = vp.shape[0] / 4, vp.shape[1] - r_obs
y, x = np.ogrid[-a : vp.shape[0] - a, -b : vp.shape[1] - b]
vp[x * x + y * y <= r_obs * r_obs] = v_obj
nx = domain_dims[0]
nz = domain_dims[1]
wall = round(nx * 0.02)
offs = round(wall / 2)
a = round((nx - wall) / 2)
b = round((nz - wall) / 2)
offs = round(wall / 2)
x = np.arange(0, vp.shape[0])
y = np.arange(0, vp.shape[1])
if obstacle:
    r = vp.shape[0] / 100
    ox = np.arange(offs, 2 * a + offs + 1, 2 * a / 50)
    oy = np.sqrt(1 - (ox - a - offs) ** 2 / a**2) * b + offs + b
    for oxx, oyy in tqdm.tqdm(zip(ox, oy)):
        mask = (y[np.newaxis, :] - oyy) ** 2 + (
            x[:, np.newaxis] - oxx
        ) ** 2 < r**2
        vp[mask] = v_wall
mask = (y[np.newaxis, :] - offs - b) ** 2 / b**2 + (
    x[:, np.newaxis] - offs - a
) ** 2 / a**2 > 1
vp[mask] = v_wall
vp[offs:-offs, :b] = v_env

In [ ]:
sonar = Sonar(
    domain_size,
    f0,
    v_env,
    vp,
    space_order=space_order,
    dt=dt,
    spatial_dist=spatial_dist,
)

sonar.set_source()
sonar.set_receiver()
sonar.finalize()

In [ ]:
plot_velocity(sonar.model, source=sonar.src.coordinates.data, receiver=sonar.rec.coordinates.data)

In [ ]:
ideal_signal = sonar.src.signal_packet

In [ ]:
recordings = {}
for a in angles:
    sonar.run_beam(a)
    recordings[a] = copy.deepcopy(sonar.rec.data)

In [ ]:
cords = np.zeros((np.size(angles), 2))
for a, v in recordings.items():
    coordinates = np.zeros((128, 2))
    for i in range(128):
        start_time = np.argmax(recordings[a][:5000, i])
        correlate = np.correlate(recordings[a][5000:, i], ideal_signal, mode="same")
        peak = 5000 + correlate.argmax()
        distance = (
            peak - start_time
        ) * sonars[a].model.critical_dt * v_env / 2
        rec_coords = sonars[a].rec.coordinates.data[i]
        coordinates[i, 0] = rec_coords[0] - distance * np.cos(np.deg2rad(a))
        coordinates[i, 1] = rec_coords[1] + distance * np.sin(np.deg2rad(a))
        cords[a // 15 - 2, :] = np.mean(coordinates, axis=0)
        

In [ ]:
plot_velocity(sonar.model, sonar.src.coordinates.data, sonar.rec.coordinates.data, cords)

### Snapshot

In [ ]:
domain_size = (60, 30)
v_env = 1.5
source_distance = 0.002
ns = 128
cy = (ns - 1) / 2 * source_distance + source_distance
f0 = 15
space_order = 8
spatial_dist = round(v_env / f0 / 3, 5)
dt = spatial_dist / 20


In [ ]:
obstacle = True
v_wall = 5.64
v_obj = 3.24
domain_dims = (
    round(domain_size[0] / spatial_dist),
    round(domain_size[1] / spatial_dist),
)
vp = np.full(domain_dims, v_env, dtype=np.float32)
r_obs = vp.shape[0] / 20
a, b = vp.shape[0] / 4, vp.shape[1] - r_obs
y, x = np.ogrid[-a : vp.shape[0] - a, -b : vp.shape[1] - b]
vp[x * x + y * y <= r_obs * r_obs] = v_obj
nx = domain_dims[0]
nz = domain_dims[1]
wall = round(nx * 0.02)
offs = round(wall / 2)
a = round((nx - wall) / 2)
b = round((nz - wall) / 2)
offs = round(wall / 2)
x = np.arange(0, vp.shape[0])
y = np.arange(0, vp.shape[1])
if obstacle:
    r = vp.shape[0] / 100
    ox = np.arange(offs, 2 * a + offs + 1, 2 * a / 50)
    oy = np.sqrt(1 - (ox - a - offs) ** 2 / a**2) * b + offs + b
    for oxx, oyy in tqdm.tqdm(zip(ox, oy)):
        mask = (y[np.newaxis, :] - oyy) ** 2 + (
            x[:, np.newaxis] - oxx
        ) ** 2 < r**2
        vp[mask] = v_wall
mask = (y[np.newaxis, :] - offs - b) ** 2 / b**2 + (
    x[:, np.newaxis] - offs - a
) ** 2 / a**2 > 1
vp[mask] = v_wall
vp[offs:-offs, :b] = v_env

In [ ]:
sonar = Sonar(
    domain_size,
    f0,
    v_env,
    vp,
    source_distance=source_distance,
    ns=ns,
    space_order=space_order,
    dt=dt,
    spatial_dist=spatial_dist,
)
sonar.set_source()
sonar.set_receiver()
sonar.finalize(snapshot_delay=0.1)

In [ ]:
plot_velocity(
    sonar.model, source=sonar.src.coordinates.data, receiver=sonar.rec.coordinates.data
)

In [ ]:
sonar.run_beam(45)
recording = copy.deepcopy(sonar.rec.data)

In [ ]:
plot_snapshot_and_signal(
    sonar.usave.data,
    recording[:, 64],
    sonar.model,
    f"output/snapshot.gif",
    sonar.src.coordinates.data,
    sonar.rec.coordinates.data,
)